# Bagging (Bootstrap aggregating)

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn import tree

Load datasets manuseados

In [2]:
df_train = pd.read_csv('../../datasets_manuseados/train_radiomics_hipocamp_treated.csv')
df_test = pd.read_csv('../../datasets_manuseados/test_radiomics_hipocamp_treated.csv')

Float64/Int64 to Float32/Int32

In [3]:
float_features = df_train.select_dtypes(include='float')
int_features = df_train.select_dtypes(include='int')
df_train[float_features.columns] = df_train[float_features.columns].astype(np.float32)
df_train[int_features.columns] = df_train[int_features.columns].astype(np.int32)
df_train.info()
print("--------------------")
float_features = df_test.select_dtypes(include='float')
int_features = df_test.select_dtypes(include='int')
df_test[float_features.columns] = df_test[float_features.columns].astype(np.float32)
df_test[int_features.columns] = df_test[int_features.columns].astype(np.int32)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 1898 entries, RowId to Transition
dtypes: float32(1896), int32(1), object(1)
memory usage: 2.2+ MB
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 1897 entries, RowId to Age
dtypes: float32(1896), int32(1)
memory usage: 741.1 KB


Model training and predict

In [ ]:
X = df_train.drop('Transition', axis=1)
y = df_train['Transition']

X = X.drop('RowId', axis=1)
X_test_sss = df_test.drop('RowId', axis=1)

X_train_sss, X_val_sss, y_train_sss, y_val_sss = train_test_split(X, y, test_size=0.2, random_state=2024)

dt = DecisionTreeClassifier(max_depth=5, max_leaf_nodes= 15, min_samples_leaf=5, min_samples_split=15, random_state=2024)
dt.fit(X_train_sss, y_train_sss)

sss = StratifiedShuffleSplit(n_splits=10, test_size=20, random_state=2024)

bg_model = BaggingClassifier(estimator=dt, bootstrap= True)

parameters = {'n_estimators': [160]}

grid_bg = GridSearchCV(estimator=bg_model, param_grid=parameters, cv=sss)
grid_bg.fit(X_train_sss, y_train_sss)
bst_bg = grid_bg.best_estimator_

bst_bg.fit(X_train_sss, y_train_sss)

bg_predictions = bst_bg.predict(X_test_sss)

op_bg = pd.DataFrame(df_test['RowId'])
op_bg['Transition'] = bg_predictions
op_bg.to_csv("../../predictions/bg.csv", index=False)